In [7]:
%load_ext autoreload

from OpenGL.GL import *
from OpenGL.GLU import *
import pygame
from pygame.locals import *
from abc import ABC, abstractmethod
from typing import Tuple, Any

In [5]:
Vec3 = Tuple[float, float, float]
Vec2 = Tuple[float, float]


class Drawable(ABC):
    def __init__(self, position: Vec3 = (0.0, 0.0, 0.0)):
        self.position: Vec3 = position

    @abstractmethod
    def draw(self) -> None:
        """Perform OpenGL drawing ."""
        ...

    @abstractmethod
    def collision_box(self) -> Any:
        """Return an implementation-defined collision box/shape in world coords."""
        ...

    @abstractmethod
    def collides_with(self, other: "Drawable") -> bool:
        """Return True if this object's collision box intersects `other`."""
        ...


class Player(Drawable):
    def draw(self):
        # --- Head (cube) ---
        # head bounds
        hx0, hx1 = -1.0, 1.0
        hy0, hy1 = 0.0, 1.0
        hz0, hz1 = -1.0, 1.0

        # skin color
        glColor3f(1.0, 0.85, 0.7)
        glBegin(GL_QUADS)
        # front (z = hz1)
        glVertex3f(hx0, hy0, hz1)
        glVertex3f(hx1, hy0, hz1)
        glVertex3f(hx1, hy1, hz1)
        glVertex3f(hx0, hy1, hz1)
        # back (z = hz0)
        glVertex3f(hx1, hy0, hz0)
        glVertex3f(hx0, hy0, hz0)
        glVertex3f(hx0, hy1, hz0)
        glVertex3f(hx1, hy1, hz0)
        # left (x = hx0)
        glVertex3f(hx0, hy0, hz0)
        glVertex3f(hx0, hy0, hz1)
        glVertex3f(hx0, hy1, hz1)
        glVertex3f(hx0, hy1, hz0)
        # right (x = hx1)
        glVertex3f(hx1, hy0, hz1)
        glVertex3f(hx1, hy0, hz0)
        glVertex3f(hx1, hy1, hz0)
        glVertex3f(hx1, hy1, hz1)
        # top
        glVertex3f(hx0, hy1, hz1)
        glVertex3f(hx1, hy1, hz1)
        glVertex3f(hx1, hy1, hz0)
        glVertex3f(hx0, hy1, hz0)
        # bottom (neck area)
        glVertex3f(hx0, hy0, hz0)
        glVertex3f(hx1, hy0, hz0)
        glVertex3f(hx1, hy0, hz1)
        glVertex3f(hx0, hy0, hz1)
        glEnd()

        # --- Eyes (on front face, slightly in front to avoid z-fighting) ---
        # white eye plates
        eye_z = hz1 + 0.01
        # left white
        glColor3f(1, 1, 1)
        glBegin(GL_QUADS)
        glVertex3f(-0.5, 0.65, eye_z)
        glVertex3f(-0.15, 0.65, eye_z)
        glVertex3f(-0.15, 0.9, eye_z)
        glVertex3f(-0.5, 0.9, eye_z)
        glEnd()
        # right white
        glBegin(GL_QUADS)
        glVertex3f(0.15, 0.65, eye_z)
        glVertex3f(0.5, 0.65, eye_z)
        glVertex3f(0.5, 0.9, eye_z)
        glVertex3f(0.15, 0.9, eye_z)
        glEnd()

        # black pupils (big & cute)
        pupil_z = hz1 + 0.02
        glColor3f(0, 0, 0)
        glBegin(GL_QUADS)
        # left pupil
        glVertex3f(-0.43, 0.69, pupil_z)
        glVertex3f(-0.22, 0.69, pupil_z)
        glVertex3f(-0.22, 0.86, pupil_z)
        glVertex3f(-0.43, 0.86, pupil_z)
        # right pupil
        glVertex3f(0.22, 0.69, pupil_z)
        glVertex3f(0.43, 0.69, pupil_z)
        glVertex3f(0.43, 0.86, pupil_z)
        glVertex3f(0.22, 0.86, pupil_z)
        glEnd()

        # small mouth
        mouth_z = hz1 + 0.02
        glColor3f(0.6, 0.15, 0.15)
        glBegin(GL_QUADS)
        glVertex3f(-0.25, 0.35, mouth_z)
        glVertex3f(0.25, 0.35, mouth_z)
        glVertex3f(0.25, 0.45, mouth_z)
        glVertex3f(-0.25, 0.45, mouth_z)
        glEnd()

        # --- Body (shirt) ---
        bx0, bx1 = -1.2, 1.2
        by0, by1 = -1.5, 0.0
        bz0, bz1 = -0.6, 0.6
        # shirt color (Steve-ish blue)
        glColor3f(0.2, 0.45, 0.85)
        glBegin(GL_QUADS)
        # front
        glVertex3f(bx0, by0, bz1)
        glVertex3f(bx1, by0, bz1)
        glVertex3f(bx1, by1, bz1)
        glVertex3f(bx0, by1, bz1)
        # back
        glVertex3f(bx1, by0, bz0)
        glVertex3f(bx0, by0, bz0)
        glVertex3f(bx0, by1, bz0)
        glVertex3f(bx1, by1, bz0)
        # left
        glVertex3f(bx0, by0, bz0)
        glVertex3f(bx0, by0, bz1)
        glVertex3f(bx0, by1, bz1)
        glVertex3f(bx0, by1, bz0)
        # right
        glVertex3f(bx1, by0, bz1)
        glVertex3f(bx1, by0, bz0)
        glVertex3f(bx1, by1, bz0)
        glVertex3f(bx1, by1, bz1)
        # top (neck area)
        glVertex3f(bx0, by1, bz1)
        glVertex3f(bx1, by1, bz1)
        glVertex3f(bx1, by1, bz0)
        glVertex3f(bx0, by1, bz0)
        # bottom
        glVertex3f(bx0, by0, bz0)
        glVertex3f(bx1, by0, bz0)
        glVertex3f(bx1, by0, bz1)
        glVertex3f(bx0, by0, bz1)
        glEnd()

        # --- Arms (sleeves) ---
        ax_in, ax_out = -1.2, -1.6
        # left arm
        glColor3f(0.2, 0.45, 0.85)
        glBegin(GL_QUADS)
        # front
        glVertex3f(ax_out, -0.5, bz1)
        glVertex3f(ax_in, -0.5, bz1)
        glVertex3f(ax_in, 0.2, bz1)
        glVertex3f(ax_out, 0.2, bz1)
        # back
        glVertex3f(ax_in, -0.5, bz0)
        glVertex3f(ax_out, -0.5, bz0)
        glVertex3f(ax_out, 0.2, bz0)
        glVertex3f(ax_in, 0.2, bz0)
        # top
        glVertex3f(ax_out, 0.2, bz1)
        glVertex3f(ax_in, 0.2, bz1)
        glVertex3f(ax_in, 0.2, bz0)
        glVertex3f(ax_out, 0.2, bz0)
        # bottom
        glVertex3f(ax_out, -0.5, bz0)
        glVertex3f(ax_in, -0.5, bz0)
        glVertex3f(ax_in, -0.5, bz1)
        glVertex3f(ax_out, -0.5, bz1)
        glEnd()

        # right arm (mirror)
        ax_in_r, ax_out_r = 1.2, 1.6
        glColor3f(0.2, 0.45, 0.85)
        glBegin(GL_QUADS)
        glVertex3f(ax_in_r, -0.5, bz1)
        glVertex3f(ax_out_r, -0.5, bz1)
        glVertex3f(ax_out_r, 0.2, bz1)
        glVertex3f(ax_in_r, 0.2, bz1)

        glVertex3f(ax_out_r, -0.5, bz0)
        glVertex3f(ax_in_r, -0.5, bz0)
        glVertex3f(ax_in_r, 0.2, bz0)
        glVertex3f(ax_out_r, 0.2, bz0)

        glVertex3f(ax_in_r, 0.2, bz1)
        glVertex3f(ax_out_r, 0.2, bz1)
        glVertex3f(ax_out_r, 0.2, bz0)
        glVertex3f(ax_in_r, 0.2, bz0)

        glVertex3f(ax_in_r, -0.5, bz0)
        glVertex3f(ax_out_r, -0.5, bz0)
        glVertex3f(ax_out_r, -0.5, bz1)
        glVertex3f(ax_in_r, -0.5, bz1)
        glEnd()

        # --- Legs (pants) ---
        # left leg
        lx0, lx1 = -0.6, -0.1
        ly0, ly1 = -3.0, -1.5
        lz0, lz1 = -0.5, 0.5
        glColor3f(0.08, 0.2, 0.6)  # dark blue pants
        glBegin(GL_QUADS)
        # front
        glVertex3f(lx0, ly0, lz1)
        glVertex3f(lx1, ly0, lz1)
        glVertex3f(lx1, ly1, lz1)
        glVertex3f(lx0, ly1, lz1)
        # back
        glVertex3f(lx1, ly0, lz0)
        glVertex3f(lx0, ly0, lz0)
        glVertex3f(lx0, ly1, lz0)
        glVertex3f(lx1, ly1, lz0)
        # left
        glVertex3f(lx0, ly0, lz0)
        glVertex3f(lx0, ly0, lz1)
        glVertex3f(lx0, ly1, lz1)
        glVertex3f(lx0, ly1, lz0)
        # right
        glVertex3f(lx1, ly0, lz1)
        glVertex3f(lx1, ly0, lz0)
        glVertex3f(lx1, ly1, lz0)
        glVertex3f(lx1, ly1, lz1)
        # top
        glVertex3f(lx0, ly1, lz1)
        glVertex3f(lx1, ly1, lz1)
        glVertex3f(lx1, ly1, lz0)
        glVertex3f(lx0, ly1, lz0)
        # bottom
        glVertex3f(lx0, ly0, lz0)
        glVertex3f(lx1, ly0, lz0)
        glVertex3f(lx1, ly0, lz1)
        glVertex3f(lx0, ly0, lz1)
        glEnd()

        # right leg
        rx0, rx1 = 0.1, 0.6
        glColor3f(0.08, 0.2, 0.6)
        glBegin(GL_QUADS)
        glVertex3f(rx0, ly0, lz1)
        glVertex3f(rx1, ly0, lz1)
        glVertex3f(rx1, ly1, lz1)
        glVertex3f(rx0, ly1, lz1)

        glVertex3f(rx1, ly0, lz0)
        glVertex3f(rx0, ly0, lz0)
        glVertex3f(rx0, ly1, lz0)
        glVertex3f(rx1, ly1, lz0)

        glVertex3f(rx0, ly0, lz0)
        glVertex3f(rx0, ly0, lz1)
        glVertex3f(rx0, ly1, lz1)
        glVertex3f(rx0, ly1, lz0)

        glVertex3f(rx1, ly0, lz1)
        glVertex3f(rx1, ly0, lz0)
        glVertex3f(rx1, ly1, lz0)
        glVertex3f(rx1, ly1, lz1)

        glVertex3f(rx0, ly1, lz1)
        glVertex3f(rx1, ly1, lz1)
        glVertex3f(rx1, ly1, lz0)
        glVertex3f(rx0, ly1, lz0)

        glVertex3f(rx0, ly0, lz0)
        glVertex3f(rx1, ly0, lz0)
        glVertex3f(rx1, ly0, lz1)
        glVertex3f(rx0, ly0, lz1)
        glEnd()

    def collision_box(self):
        return []

    def collides_with(self, other):
        return False
        
    # def getDirection(self):
        
        
        

In [6]:
def draw_ground():
    glBegin(GL_QUADS)
    glColor3f(0.3, 0.3, 0.3)

    glVertex3f(-5, -0.01, -5)
    glVertex3f(5, -0.01, -5)
    glVertex3f(5, -0.01, 5)
    glVertex3f(-5, -0.01, 5)

    glEnd()


player = Player()
pygame.init()

display = (800, 600)
pygame.display.set_mode(display, DOUBLEBUF | OPENGL)
pygame.display.set_caption("My OpenGL Window")

glMatrixMode(GL_PROJECTION)
glLoadIdentity()
gluPerspective(45, 800 / 600, 0.1, 50.0)

glMatrixMode(GL_MODELVIEW)
glLoadIdentity()
# glTranslatef(0, 0, -5)
# fmt: off
gluLookAt(
    0, 3, 5, # camera xyz
    0, 0, 0, # where to look
    0, 1, 0, # direction
)
# fmt: on
glEnable(GL_DEPTH_TEST)

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False

    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)

    draw_ground()
    glPushMatrix()
    glTranslatef(0, 1, 0)
    glScalef(0.2, 0.2, 0.2)
    player.draw()

    glPopMatrix()

    pygame.display.flip()
    pygame.time.wait(10)  # ms

pygame.quit()

In [68]:
pygame.quit()